# Zensus 2022

# Einführung
Dieses Notebook beschreibt den ETL-Prozess zur Verarbeitung und Übertragung von Zensusdaten in eine PostgreSQL-Datenbank. Die Daten werden aus einer Excel-Datei extrahiert, bereinigt und in ein Sternschema transformiert. Ziel ist es, eine Datenbankstruktur zu schaffen, die effiziente analytische Abfragen unterstützt.


## Import und Setup
In diesem Abschnitt werden die benötigten Python-Bibliotheken importiert und eine Verbindung zur PostgreSQL-Datenbank hergestellt. Dies ermöglicht uns die spätere Speicherung der verarbeiteten Daten.


In [116]:
import pandas as pd
import os
import sqlalchemy as sql
import numpy as np
import sqlite3
import psycopg2
from sqlalchemy import create_engine

In [117]:
# Deine Datenbankverbindungseinstellungen
db_user = 'postgres'
db_password = 'neues_passwort'
db_host = 'localhost'
db_port = '5432'
db_name = 'postgres'

# Verbindung zur PostgreSQL-Datenbank herstellen
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


# Datenextraktion (Extract)
Hier werden die Daten aus der Excel-Datei extrahiert. Jede Tabelle (Blatt) in der Datei, die mit "CSV" beginnt, wird in einen Pandas DataFrame geladen. Dies ermöglicht uns eine einfache Verarbeitung und Transformation der Daten in den folgenden Schritten.


In [118]:
# Pfad zur Excel-Datei
file_path = 'C:\\Users\\hp\\OneDrive\\Desktop\\DBU\\SP3-1.Dateninfrastrukturen\\Pruefung\\Zensus\\input\\Zensus.xlsx'
# Lese die Excel-Datei ein und speichere die Blätter als einzelne DataFrames
sheets = pd.read_excel(file_path, sheet_name=None, dtype={'_RS': str})
data_frames = {sheet_name: data for sheet_name, data in sheets.items() if sheet_name.startswith('CSV')} # nur die Blätter mit 'CSV' im Namen

# check
print("Dateninfrastruktur aufgesetzt und Excel-Daten geladen.")

Dateninfrastruktur aufgesetzt und Excel-Daten geladen.


# Datenbereinigung (Transform)
In diesem Schritt werden unerwünschte Zeichen wie `-`, `/`, `.` und `–` durch `0` ersetzt, um die Daten zu bereinigen. Außerdem wird die Spalte `Regionalebene` von Leerzeichen und Sonderzeichen bereinigt, um eine konsistente Datenbasis zu gewährleisten.


In [119]:
# Speichern der DataFrames in separaten Variablen mit angepassten Namen
created_dfs = {}
for sheet_name, data in data_frames.items():
    # Ersetzen von '-' und '/' durch '_' in den Tabellenblätternamen
    new_sheet_name = sheet_name.replace('-', '_').replace('/', '_')
    
    # Ersetzen von '.', '/', '-' und '–' durch 0 in den Daten
    df = data.apply(lambda x: x.map(lambda y: 0 if y in ['.', '/', '-', '–'] else y))
    
    # Bereinigung der 'Regionalebene'-Spalte (Kreise)
    if 'Regionalebene' in df.columns:
        df['Regionalebene'] = df['Regionalebene'].str.replace('/', '')
        df['Regionalebene'] = df['Regionalebene'].str.replace(' ', '')
    # Umwandlung in numerische Typen für Spalten ab der 5.
    for col in df.columns[4:]:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    
    # Spaltennamen in Kleinbuchstaben ändern
    df.columns = [col.lower() for col in df.columns]

    globals()[new_sheet_name] = df
    created_dfs[new_sheet_name] = df

# Anzeigen der neuen Namen der erstellten DataFrames
print("Erstellte DataFrames:")
for df_name in created_dfs.keys():
    print(df_name)

Erstellte DataFrames:
CSV_Klassenstufe
CSV_Schulform
CSV_Hoechster_Schulabschluss
CSV_Hoechster_berufl_Abschluss
CSV_Erwerbsstatus
CSV_ET_Alter
CSV_ET_Hoechst_berufl_Abschl
CSV_ET_Stellung_im_Beruf
CSV_ET_Beruf_Hauptgr_ISCO08
CSV_ET_Wirtschaftszweig


In [120]:
CSV_ET_Alter['_rs'].dtype

dtype('O')

In [121]:
CSV_Klassenstufe.groupby('regionalebene')['schueler_klss_stp'].sum()

regionalebene
Bund                                  8903780
Gemeinde                              6631640
Gemeindeverband                       1310750
Land                                  8903780
Regierungsbezirk                      7013880
StadtkreiskreisfreieStadtLandkreis    8903770
Name: schueler_klss_stp, dtype: int64

## Erstellung und Bereinigung der Stadt-Typen Tabelle

In diesem Abschnitt wird eine neue Tabelle für `stadt_typen` erstellt, die Informationen über die verschiedenen Stadtarten (z.B. Stadt, Landeshauptstadt, Hansestadt) enthält. Die Stadtarten werden aus den bestehenden DataFrames extrahiert und in einer separaten Tabelle organisiert.

In [122]:
# Neue Tabelle für stadt-Typen erstellen
stadt_typen = pd.DataFrame(columns=['schluessel', 'typ'])

# Bereinigung der DataFrames und Auslagern der stadt-Typen
for name, df in created_dfs.items():
    # Extrahieren des Stadtnamens und Typs
    df['stadt'] = df['name'].str.split(',').str[0]
    df['typ'] = df['name'].str.split(',').str[1].str.strip()
    
    # Kombinieren in die neue Tabelle
    temp_df = df[['_rs', 'typ']].drop_duplicates().rename(columns={'_rs': 'schluessel'})
    stadt_typen = pd.concat([stadt_typen, temp_df])
    
    # Bereinigen der 'name'-Spalte
    df['name'] = df['stadt']
    
    # Entfernen der temporären Spalten
    df = df.drop(columns=['stadt', 'typ'])
    
    # Aktualisieren des DataFrames im Dictionary
    created_dfs[name] = df

# Entfernen von Duplikaten und NaN-Werten in der neuen Tabelle
stadt_typen = stadt_typen.drop_duplicates().dropna().reset_index(drop=True)
stadt_typen['schluessel'] = stadt_typen['schluessel'].astype('str')  # Umwandlung in String
# Neue Tabelle und bereinigte DataFrames anzeigen
print("stadt-Typen Tabelle:")
print(stadt_typen.head())

print("Bereinigte DataFrames:")
for name, df in created_dfs.items():
    print(f"{name}:")
    print(df.head())

stadt-Typen Tabelle:
     schluessel               typ
0         01001             Stadt
1  010010000000             Stadt
2         01002  Landeshauptstadt
3  010020000000  Landeshauptstadt
4         01003        Hansestadt
Bereinigte DataFrames:
CSV_Klassenstufe:
   berichtszeitpunkt           _rs                name  \
0           20220515            00         Deutschland   
1           20220515            01  Schleswig-Holstein   
2           20220515         01001           Flensburg   
3           20220515  010010000000           Flensburg   
4           20220515         01002                Kiel   

                        regionalebene  schueler_klss_stp  \
0                                Bund            8903780   
1                                Land             320060   
2  StadtkreiskreisfreieStadtLandkreis              10380   
3                            Gemeinde              10380   
4  StadtkreiskreisfreieStadtLandkreis              24340   

   schueler_klss_stp__m 

## Schlüsselspalten (regionalebenen) erstellen
In diesem Abschnitt werden verschiedene Schlüsselspalten aus vorhandenen Daten extrahiert, um sie später als Primärschlüssel in den Dimensionstabellen zu verwenden. Dieser Schritt ist wichtig, um die Daten in die zweite Normalform zu bringen.


In [123]:
# Definieren der Regionalebenen
liste = [
    'Gemeinde',
    'Gemeindeverband',
    'Land',
    'Regierungsbezirk',
    'StadtkreiskreisfreieStadtLandkreis'
]

In [124]:
# Ein leeres Dictionary regionalebenen wird erstellt, um die DataFrames für jede Regionalebene zu speichern.
regionalebenen = {}
for ebene in liste: # Iteration über die Regionalebenen
    regionalebenen[ebene] = pd.DataFrame() # Für jede Ebene wird ein leerer DataFrame im Dictionary regionalebenen erstellt.
    for df in created_dfs.values():
        if 'regionalebene' in df.columns: # Innerhalb der Schleife über die DataFrames wird überprüft, ob die Spalte 'regionalebene' vorhanden ist.
            filtered_df = df[df['regionalebene'] == ebene][['_rs', 'name']] # Zeilen werden gefiltert und nur die Spalten '_rs' und 'name' behalten.
            regionalebenen[ebene] = pd.concat([regionalebenen[ebene], filtered_df], ignore_index=True)# Die gefilterten Zeilen werden an die dfs angehängt.
    regionalebenen[ebene] = regionalebenen[ebene].drop_duplicates().reset_index(drop=True) # Duplikate werden entfernt und der Index zurückgesetzt.

    # check
    print(f"Tabelle für {ebene} wurde erfolgreich erstellt.")
    print(regionalebenen[ebene].head())

Tabelle für Gemeinde wurde erfolgreich erstellt.
            _rs         name
0  010010000000    Flensburg
1  010020000000         Kiel
2  010030000000       Lübeck
3  010040000000   Neumünster
4  010510011011  Brunsbüttel
Tabelle für Gemeindeverband wurde erfolgreich erstellt.
         _rs                    name
0  010515163  Burg-St. Michaelisdonn
1  010515166           Marne-Nordsee
2  010515169                   Eider
3  010515172           Heider Umland
4  010515175      Mitteldithmarschen
Tabelle für Land wurde erfolgreich erstellt.
  _rs                 name
0  01   Schleswig-Holstein
1  02              Hamburg
2  03        Niedersachsen
3  04               Bremen
4  05  Nordrhein-Westfalen
Tabelle für Regierungsbezirk wurde erfolgreich erstellt.
   _rs                              name
0  031  Statistische Region Braunschweig
1  032      Statistische Region Hannover
2  033      Statistische Region Lüneburg
3  034     Statistische Region Weser-Ems
4  051              Reg.-Bez. 

In [125]:
regionalebenen['Regierungsbezirk']

,_rs,name
0,031,Statistische Region Braunschweig
1,032,Statistische Region Hannover
2,033,Statistische Region Lüneburg
3,034,Statistische Region Weser-Ems
4,051,Reg.-Bez. Düsseldorf
5,053,Reg.-Bez. Köln
6,055,Reg.-Bez. Münster
7,057,Reg.-Bez. Detmold
8,059,Reg.-Bez. Arnsberg
9,064,Reg.-Bez. Darmstadt


In [126]:
# Erstellen der Spalten für Gemeindeverbandsschlüssel (erste 9 Stellen)
for ebene in ['Gemeinde', 'Gemeindeverband']:
    regionalebenen[ebene]['gemeindeverbandsschluessel'] = regionalebenen[ebene]['_rs'].astype(str).str[:9]

# Erstellen der Spalten für Kreisschlüssel (erste 5 Stellen)
for ebene in ['Gemeinde','Gemeindeverband', 'StadtkreiskreisfreieStadtLandkreis']:
    regionalebenen[ebene]['kreisschluessel'] = regionalebenen[ebene]['_rs'].astype(str).str[:5]

# Erstellen der Spalten für Landesschlüssel (erste 3 Stellen)
for ebene in ['Gemeinde','StadtkreiskreisfreieStadtLandkreis', 'Regierungsbezirk']:
    regionalebenen[ebene]['regbezirkschluessel'] = regionalebenen[ebene]['_rs'].astype(str).str[:3]

# Erstellen der Spalten für Landesschlüssel (erste 2 Stellen)
for ebene in ['Gemeinde','Gemeindeverband','Regierungsbezirk', 'Land']:
    regionalebenen[ebene]['landesschluessel'] = regionalebenen[ebene]['_rs'].astype(str).str[:2]


# Alle Tabellen ohne die Spalte '_rs' speichern (ausser Gemeinde)
for ebene, df in regionalebenen.items():
    if ebene != 'Gemeinde':
        regionalebenen[ebene] = df.drop('_rs', axis=1)

# _rs in Gemeindeschlüssel umbenennen
regionalebenen['Gemeinde'] = regionalebenen['Gemeinde'].rename(columns={'_rs': 'gemeindeschluessel'})

In [127]:
# Überprüfen der ersten Zeilen der Tabellen

print("Gemeindetabelle:")
print(regionalebenen['Gemeinde'].head())

print("\nGemeindeverbandstabelle:")
print(regionalebenen['Gemeindeverband'].head())

print("\nKreis:")
print(regionalebenen['StadtkreiskreisfreieStadtLandkreis'].head())

print("\nRegBezirk:")
print(regionalebenen['Regierungsbezirk'].head())

print("\nLändertabelle:")
print(regionalebenen['Land'].head())

Gemeindetabelle:
  gemeindeschluessel         name gemeindeverbandsschluessel kreisschluessel  \
0       010010000000    Flensburg                  010010000           01001   
1       010020000000         Kiel                  010020000           01002   
2       010030000000       Lübeck                  010030000           01003   
3       010040000000   Neumünster                  010040000           01004   
4       010510011011  Brunsbüttel                  010510011           01051   

  regbezirkschluessel landesschluessel  
0                 010               01  
1                 010               01  
2                 010               01  
3                 010               01  
4                 010               01  

Gemeindeverbandstabelle:
                     name gemeindeverbandsschluessel kreisschluessel  \
0  Burg-St. Michaelisdonn                  010515163           01051   
1           Marne-Nordsee                  010515166           01051   
2             

## Erstellung und Anpassung der Tabellen für die weiteren Daten
In diesem Abschnitt werden die Tabellen für die in tabellen_liste definierten Daten erstellt und die Spalten angepasst.

In [128]:
# Liste erstellen fuer die uebrigen dfs
tabellen_liste = [
    'CSV_Klassenstufe',
    'CSV_Schulform',
    'CSV_Hoechster_Schulabschluss',
    'CSV_Hoechster_berufl_Abschluss',
    'CSV_Erwerbsstatus',
    'CSV_ET_Alter',
    'CSV_ET_Hoechst_berufl_Abschl',
    'CSV_ET_Stellung_im_Beruf',
    'CSV_ET_Beruf_Hauptgr_ISCO08',
    'CSV_ET_Wirtschaftszweig'
]

In [129]:
# Ein leeres Dictionary andere_tabellen wird erstellt, um die bereinigten DataFrames zu speichern.
andere_tabellen = {}
# Iteration über die Liste der Tabellen
for tabelle in tabellen_liste:
    tabelle_name = tabelle.replace('CSV_', '').lower()  # Ersetzen von 'CSV_' durch '' und Umwandlung in Kleinbuchstaben
    df = created_dfs[tabelle].drop(columns=['berichtszeitpunkt', 'name', 'regionalebene']).drop_duplicates().reset_index(drop=True)  # Entfernen der Spalten
    
    # Eine neue Spalte schluessel wird erstellt, die den Inhalt der Spalte _rs als String-Wert speichert.
    df['schluessel'] = df['_rs'].astype(str) 
    df = df.drop(columns=['_rs']) # Entfernen der Spalte _rs
    df = df.fillna(0)  # NA-Werte durch 0 ersetzen
    
    # Verschieben der schluessel-Spalte an die erste Position
    cols = ['schluessel'] + [col for col in df if col != 'schluessel']
    df = df[cols]
    andere_tabellen[tabelle_name] = df  # Verwendung von tabelle_name statt tabelle

# Anzeigen aller Tabellennamen in andere_tabellen
print("Alle Tabellennamen in andere_tabellen:")
for table_name in andere_tabellen.keys():
    print(table_name)

Alle Tabellennamen in andere_tabellen:
klassenstufe
schulform
hoechster_schulabschluss
hoechster_berufl_abschluss
erwerbsstatus
et_alter
et_hoechst_berufl_abschl
et_stellung_im_beruf
et_beruf_hauptgr_isco08
et_wirtschaftszweig


In [130]:
for tabelle in andere_tabellen:
    print(f"Datentypen für Tabelle '{tabelle}':")
    print(andere_tabellen[tabelle].dtypes)
    print("\n")

Datentypen für Tabelle 'klassenstufe':
schluessel                object
schueler_klss_stp          int64
schueler_klss_stp__m       int64
schueler_klss_stp__w       int64
schueler_klss_stp__1       int64
schueler_klss_stp__1_m     int64
schueler_klss_stp__1_w     int64
schueler_klss_stp__2       int64
schueler_klss_stp__2_m     int64
schueler_klss_stp__2_w     int64
schueler_klss_stp__3       int64
schueler_klss_stp__3_m     int64
schueler_klss_stp__3_w     int64
dtype: object


Datentypen für Tabelle 'schulform':
schluessel               object
schueler_sch_stp          int64
schueler_sch_stp__m       int64
schueler_sch_stp__w       int64
schueler_sch_stp__1       int64
schueler_sch_stp__1_m     int64
schueler_sch_stp__1_w     int64
schueler_sch_stp__2       int64
schueler_sch_stp__2_m     int64
schueler_sch_stp__2_w     int64
schueler_sch_stp__3       int64
schueler_sch_stp__3_m     int64
schueler_sch_stp__3_w     int64
schueler_sch_stp__4       int64
schueler_sch_stp__4_m     int64


# Übertragung in die Datenbank (Load)
Die bereinigten und transformierten DataFrames werden in die PostgreSQL-Datenbank übertragen.

In [131]:

# Andere Tabellen in die PostgreSQL-Datenbank übertragen
for tabelle, df in andere_tabellen.items():
    df.to_sql(f'dim_{tabelle.lower()}', engine, if_exists='replace', index=False)

# Regionalebenen in die PostgreSQL-Datenbank übertragen
for ebene, df in regionalebenen.items():
    df.to_sql(f'dim_{ebene.lower()}', engine, if_exists='replace', index=False)

# Stadt_Typen in die PostgreSQL-Datenbank übertragen
stadt_typen.to_sql('dim_stadt_typen', engine, if_exists='replace', index=False)

print("DataFrames erfolgreich in die PostgreSQL-Datenbank übertragen.")

DataFrames erfolgreich in die PostgreSQL-Datenbank übertragen.
